In [1]:
import cv2
import torch
import torch.nn.functional as F
from PIL import Image
from torch import nn
from nets.FreezeNet import FreezeNet
import numpy as np

from glob import glob
import random
import matplotlib.pyplot as plt
import torch.nn.functional as F
from tqdm import tqdm
import os
import pandas as pd
from utils.util import find_ring

In [2]:
def average_non_zero_pixel(gray_image):
    non_zero_indices = np.nonzero(gray_image)
    non_zero_values = gray_image[non_zero_indices]
    average_value = np.mean(non_zero_values)
    return average_value

def get_green_yello_ratio(new,mask):
    hsv=cv2.cvtColor(new,cv2.COLOR_BGR2HSV)
    lower_green = np.array([25, 45, 0])
    upper_green = np.array([180, 255, 240])
    
    green_mask = cv2.inRange(hsv, lower_green, upper_green)
    new1 = cv2.bitwise_and(new,new,mask=green_mask)
    yellow_mask=255*mask-green_mask
    yellow_hsv=cv2.bitwise_and(hsv,hsv,mask=yellow_mask)
    green_pixels = cv2.countNonZero(green_mask)
    total_pixels = cv2.countNonZero(mask)
    h,s,v=cv2.split(hsv)
    h_value=average_non_zero_pixel(h)
    s_value=average_non_zero_pixel(s)
    h,s,v=cv2.split(yellow_hsv)
    h_y=average_non_zero_pixel(h)
    s_y=average_non_zero_pixel(s)
    if total_pixels==0:
        return 0
    ratio_green = green_pixels / total_pixels
    return ratio_green,green_pixels,total_pixels-green_pixels,h_value,s_value,h_y,s_y


In [3]:

input_shape=(512,512)
model = FreezeNet()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.load_state_dict(torch.load('model/phoneLite.pth', map_location=device))

model = nn.DataParallel(model)
model = model.to(device)
model = model.eval()

In [5]:
input_shape=(512,512)
content='Plot,GreenLeafRation,YellowLeafRation,GreenLeaf,YellowLeafArea,TotalLeafArea,Hue,Sat\n'
save_path='phone_phenotypes.csv'
with open(save_path,'w',encoding='utf-8') as f:
    f.write(content)
load=[]   

files=sorted(glob('datasets/*.jpg'))
with tqdm(total=len(files),desc='Progress:',postfix=dict,mininterval=0.3) as pbar:
    for index,file in enumerate(files):
        imgname=file.split('/')[-1].split('.')[0]
        image = cv2.imread(file)
        h,w=image.shape[:2]
        if h<w:
            image=cv2.rotate(image,cv2.ROTATE_90_CLOCKWISE)
            h,w=w,h
        ratio=512/w
        x1,x2,y1,y2,circles=find_ring(image.copy(),ratio)
        new=np.zeros((int(h),int(w),3),dtype=np.uint8)
        cv2.circle(new, (round(circles[0][0][0]/ratio), round(circles[0][0][1]/ratio)), round(circles[0][0][2]/ratio), (255, 255, 255), -1)
        image=cv2.bitwise_and(image,new)
        image=image[y1:y2,x1:x2,:]
        image=cv2.resize(image,(input_shape[0],input_shape[0]))
        img=torch.from_numpy(image/255).float().to(device)
        img=img.unsqueeze(0)
        img=img.permute(0,3,1,2)
        with torch.no_grad():
            output=model(img)
        output = output[0].permute(1,2,0).cpu().argmax(axis=-1).numpy().astype(np.uint8)
        new = cv2.bitwise_and(image,image,mask=output)
        r,g,y,h,s,h_y,s_y=get_green_yello_ratio(new,output)  
        with open(save_path,'a',encoding='utf-8') as f:
            f.write('{},{},{},{},{},{},{},{}\n'.format(imgname,r,1-r,g,y,g+y,h,s))
        pbar.update(1)
pd.read_csv('phone_phenotypes.csv')

Progress:: 100%|██████████| 6/6 [00:01<00:00,  3.31it/s<class 'dict'>]


,Plot,GreenLeafRation,YellowLeafRation,GreenLeaf,YellowLeafArea,TotalLeafArea,Hue,Sat
0,2022C016_20230218_102707,0.118836,0.881164,5297,39277,44574,19.848836,121.652914
1,2022C035_20230218_102926,0.064533,0.935467,2474,35863,38337,18.718496,126.385867
2,2022C071_20230218_103407,0.493738,0.506262,28699,29427,58126,25.736206,121.858546
3,2022C094_20230218_103739,0.112724,0.887276,3744,29470,33214,19.020023,134.975942
4,2022C121_20230218_104116,0.106132,0.893868,3683,31019,34702,19.538557,98.618293
5,2022C147_20230218_104450,0.123193,0.876807,3963,28206,32169,19.963008,106.759893
